In [5]:
import pandas as pd
from src.student_eval.app.evaluation import models
df = pd.DataFrame({
    'user_id'   : ['alice','alice','bob','bob']*3,
    'skill_name': ['fractions']*6 + ['equations']*6,
    'correct'   : [0,1,0,0,  1,1,  1,0,0,1,1,1],
    'item_id'   : ['A','B','A','B']*3
})
df['order_id'] = df.groupby(['user_id','skill_name']).cumcount() + 1
print(df)

   user_id skill_name  correct item_id  order_id
0    alice  fractions        0       A         1
1    alice  fractions        1       B         2
2      bob  fractions        0       A         1
3      bob  fractions        0       B         2
4    alice  fractions        1       A         3
5    alice  fractions        1       B         4
6      bob  equations        1       A         1
7      bob  equations        0       B         2
8    alice  equations        0       A         1
9    alice  equations        1       B         2
10     bob  equations        1       A         3
11     bob  equations        1       B         4


In [6]:
user_id = df['user_id'].to_list()
skill_name = df['skill_name'].to_list()
correct = df['correct'].to_list()
item_id = df['item_id'].to_list()
order_id = df['order_id'].to_list()

model=models.StudentModel()
model.csv_path = "data.csv"
model.model_path = "model.pkl"
results = model.update_dataset(order_id, user_id, skill_name, correct, item_id)
print(results["students_states"])
print(results["skills_states"])

  user_id  equations_learn  fractions_learn  equations_forget  \
1   alice          0.18204          0.14653           0.00000   
2     bob          0.73968          0.01945           0.00000   

   fractions_forget  
1           0.00000  
2           0.00000  
       skill   prior  learns  guesses   slips  forgets
0  equations 0.00000 0.63193  0.43978 0.10387  0.64113
1  fractions 0.00000 0.48813  0.02845 0.08062  0.00656


In [7]:
order_id = [1, 2, 1]
skill_name = ["inequalities", "inequalities", "inequalities"]
user_id = ["carlos", "carlos", "alice"]
item_id = ["A", "B", "A"]
correct = [1, 0, 1]
results = model.update_dataset(order_id, user_id, skill_name, correct, item_id)
print(results["students_states"])
print(results["skills_states"])

  user_id  equations_learn  fractions_learn  inequalities_learn  \
1   alice          0.18204          0.14653             0.03838   
2     bob          0.73968          0.01945                 NaN   
3  carlos              NaN              NaN             0.03041   

   equations_forget  fractions_forget  inequalities_forget  
1           0.00000           0.00000              0.00000  
2           0.00000           0.00000                  NaN  
3               NaN               NaN              0.00000  
          skill   prior  learns  guesses   slips  forgets
0     equations 0.00000 0.63193  0.43978 0.10387  0.64113
1     fractions 0.00000 0.48813  0.02845 0.08062  0.00656
2  inequalities 0.00000 0.01107  0.66256 0.19280  0.81275
